# MERGE YOLO AND CLASSIFIER FOR SUBMISSION

## IMPORTS

In [ ]:
import pandas as pd

## CONFIG

#### Project Config:

- root_dir :
    - YOLO_RESULT
    - CLASSIFIER_RESULT

In [ ]:
YOLO_RESULT = ''
CLASSIFIER_RESULT = ''
THRESHOLD = 0.05

## FUNCTIONS

In [ ]:
def read_pred():
    pred_yolo = pd.read_csv(YOLO_RESULT)
    print('YOLO: ')
    display(pred_yolo.head())
    pred_classifier = pd.read_csv(CLASSIFIER_RESULT)
    print('CLASSIFIER: ')
    display(pred_classifier.head())
    return pred_yolo, pred_classifier

In [ ]:
def merge_pred(pred_yolo, pred_classifier):
    pred = pd.merge(pred_yolo, pred_classifier, on='image_id', how='left')
    display(pred.head())
    return pred

In [ ]:
def filter_2cls(row):
    prob = row['target']
    if prob > (1-THRESHOLD):
        ## Less chance of having any disease
        row['PredictionString'] = '14 1 0 0 1 1'
    elif THRESHOLD <= prob <= (1-THRESHOLD):
        ## More change of having any diesease
        row['PredictionString'] += f' 14 {round(prob,3)} 0 0 1 1'
    elif THRESHOLD >= prob:
        ## Good chance of having any disease so believe in object detection model
        row['PredictionString'] = row['PredictionString']
    else:
        print(prob)
        raise ValueError('Prediction must be from [0-1]')
    return row

In [ ]:
def merge_2cls(pred):
    sub = pred.apply(filter_2cls, axis=1)
    display(sub.head())
    sub[['image_id', 'PredictionString']].to_csv('submission.csv', index=False)
    print('Final submission saved at submission.csv')

## RUN

#### READ PREDICTIONS

In [ ]:
pred_yolo, pred_classifier = read_pred()

#### MERGE PREDICTIONS

In [ ]:
pred = merge_pred(pred_yolo, pred_classifier)

#### CREATE SUBMISSION

In [ ]:
merge_2cls(pred)

## MERGE CLASSIFIER RESULTS

In [ ]:
file1 = ''
file2 = ''
file3 = ''
file4 = ''

df1 = pd.read_csv(file1)
df1 = df1.rename(columns={'target': 'target_1'})
df2 = pd.read_csv(file2)
df2 = df2.rename(columns={'target': 'target_2'})
df3 = pd.read_csv(file3)
df3 = df3.rename(columns={'target': 'target_3'})
df4 = pd.read_csv(file4)
df4 = df4.rename(columns={'target': 'target_4'})

df = pd.merge(df1, df2, on='image_id', how='left')
df = pd.merge(df, df3, on='image_id', how='left')
df = pd.merge(df, df4, on='image_id', how='left')

df['target'] = df[['target_1', 'target_2', 'target_3', 'target_4']].mean(axis=1)
df = df[['image_id', 'target']]

display(df.head())